In [1]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("TransitTrends-Taxi-Data-Cleaning") \
	    .config("spark.some.config.option", "some-value") \
        .config("spark.driver.extraJavaOptions", "-Dsun.net.client.defaultConnectTimeout=100000 -Dsun.net.client.defaultReadTimeout=100000") \
        .config("spark.executor.heartbeatInterval", "100000ms") \
        .config("spark.network.timeout", "12000s") \
	    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/06 18:59:11 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/05/06 18:59:12 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/05/06 18:59:12 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/05/06 18:59:12 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
#importing libraries
from pyspark.sql.functions import lit
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
import math
import time
import pyspark.sql.functions as F
from pyspark.sql.functions import concat
from pyspark.sql.functions import count,asc
import matplotlib.pyplot as plt

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/miniconda3/share/proj failed


In [3]:
# importing green cabs data, dropping unnecessary coloumns and renaming few coloumns to match yellow cabs data:
#green_cabs_df = spark.read.csv(path='/shared/TAXI_SAMPLE/GREEN_CABS/',header=True, inferSchema= True).withColumn("taxi_type", lit("green")).drop("trip_type").withColumnRenamed("lpep_dropoff_datetime", "tpep_dropoff_datetime").withColumnRenamed("lpep_pickup_datetime", "tpep_pickup_datetime")
green_cabs_df = spark.read.option("dictionaryFilterEnabled", "false").parquet("/shared/TAXI/GREEN_CABS/").withColumn("taxi_type", lit("green")).drop("trip_type").withColumnRenamed("lpep_dropoff_datetime", "tpep_dropoff_datetime").withColumnRenamed("lpep_pickup_datetime", "tpep_pickup_datetime")

# importing yellow cabs data:
#yellow_cabs_df = spark.read.csv(path='/shared/TAXI_SAMPLE/YELLOW_CABS/',header=True, inferSchema= True).withColumn("taxi_type", lit("yellow"))
yellow_cabs_df = spark.read.option("dictionaryFilterEnabled", "false").parquet("/shared/TAXI/YELLOW_CABS/").withColumn("taxi_type", lit("yellow"))

# note: sometime there might be .ipynb checkpoints in /shared/GREEN_CABS folder. remove them hdfs before running this cell
yellow_cabs_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)
 |-- taxi_type: string (nullable = false)



In [4]:
green_cabs_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- taxi_type: string (nullable = false)



In [5]:
print(green_cabs_df.count())
green_cabs_df.show(5)

9944318


+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+--------------------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|congestion_surcharge|taxi_type|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+--------------------+---------+
|       2| 2018-12-21 15:17:29|  2018-12-21 15:18:57|                 N|       1.0|         264|         264|            5.0|          0.0|        3.0|  0.5|    0.

In [6]:
print(yellow_cabs_df.count())
yellow_cabs_df.show(5)

179807942


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|taxi_type|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+
|       1| 2019-03-01 00:24:41|  2019-03-01 00:25:31|            1.0|          0.0|       1.0|                 N|         145|         145|           2|     

In [7]:
# combining yellow and green cabs data into a single dataset:
cabs_df = yellow_cabs_df.union(green_cabs_df)
# dropping unnnecessary coloumns:
cabs_df = cabs_df.drop("store_and_fwd_flag")

In [8]:
print(cabs_df.count())
cabs_df.show(5)

189752260
+--------+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|taxi_type|
+--------+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+
|       1| 2019-03-01 00:24:41|  2019-03-01 00:25:31|            1.0|          0.0|       1.0|       145.0|       145.0|         2.0|        2.5|  0.5|    0.5|       0.0|         0.0|                  0.3

In [9]:
# reading taxi zones data
taxi_zones = spark.read.csv("/shared/TAXI/taxi_zones.csv")

In [10]:
# Clean the column names
cabs_df = cabs_df.select([F.col(column).alias(column.strip()) for column in cabs_df.columns])
#Filter for data in 2019-2021, removing the last week of 2018 from the dataset
cabs_df = cabs_df.filter(F.year("tpep_pickup_datetime").isin([2019, 2020, 2021]))

In [11]:
#converting pickup and dropoff datetime to timestamp
cabs_df = cabs_df.withColumn("pickup_date_time", F.to_timestamp(cabs_df["tpep_pickup_datetime"], "MM/dd/yyyyHH:mm:ss"))
cabs_df = cabs_df.withColumn("dropoff_date_time", F.to_timestamp(cabs_df["tpep_dropoff_datetime"], "MM/dd/yyyyHH:mm:ss"))

In [12]:
#dropping unnecesary colomns
cabs_df = cabs_df.drop("tpep_pickup_datetime","tpep_dropoff_datetime")
print(cabs_df.count())
cabs_df.show(5)

149253995
+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+-------------------+-------------------+
|VendorID|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|taxi_type|   pickup_date_time|  dropoff_date_time|
+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+-------------------+-------------------+
|       1|            1.0|          0.0|       1.0|       145.0|       145.0|         2.0|        2.5|  0.5|    0.5|       0.0|         0.0|                  0.3|         3.8|                 0.0|       null|   ye

In [13]:
#cabs_df = cabs_df.withColumn("pickup_datetime", cabs_df["tpep_pickup_datetime"].cast("timestamp"))
cabs_df_1 = cabs_df.withColumn("month", F.month(cabs_df["pickup_date_time"])).withColumn("year", F.year(cabs_df["pickup_date_time"]))

In [14]:
monthly_records = cabs_df_1.select("month","year")
#cabs_df = monthly_records.withColumn("month/year", concat(F.col("month"), F.lit("/"), F.col("year"))).drop("month","year")

In [15]:
monthly_records.show(5)

+-----+----+
|month|year|
+-----+----+
|    3|2019|
|    3|2019|
|    3|2019|
|    3|2019|
|    3|2019|
+-----+----+
only showing top 5 rows



In [16]:

count_df = monthly_records.groupBy('year','month').agg(count('*').alias('count')).orderBy(asc('year'), asc('month'))
count_df.show(5)

+----+-----+-------+
|year|month|  count|
+----+-----+-------+
|2019|    1|8368407|
|2019|    2|7664760|
|2019|    3|8509420|
|2019|    4|8043806|
|2019|    5|8144006|
+----+-----+-------+
only showing top 5 rows



In [17]:


# Convert the PySpark DataFrame to Pandas DataFrame
pd_df = count_df.toPandas()

plt.figure(figsize=(30, 15))

# Create the bar plot using matplotlib
plt.bar(pd_df['year'].astype(str) + '-' + pd_df['month'].astype(str), pd_df['count'])

# Set the title and axis labels
plt.title('Trips per Year-Month')
plt.xlabel('Year-Month')
plt.ylabel('Count')
plt.savefig('trips_per_month.png' , dpi=500)
# Show the plot
plt.show()


In [19]:
taxi_zones.show(5)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:42717)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:42717)